In [1]:
import torch
import torch.nn as nn
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from copy import deepcopy
import torch.nn.functional as F
from torch.optim import AdamW
from datasets import load_dataset

# --- Classe Principale Uchida ---

class UchidaDDPM:
    def __init__(self, model_id, device="cuda"):
        self.device = device
        self.model_id = model_id

        # Chargement du modèle
        self.pipeline = DDPMPipeline.from_pretrained(model_id)
        self.unet = self.pipeline.unet.to(device)
        self.scheduler = self.pipeline.scheduler

        # Configuration par défaut
        self.config = {
            "layer_name": "mid_block.resnets.1.conv2.weight", # Cible robuste
            "watermark_len": 64,
            "lr": 1e-4,
            "lambda_wat": 1.0,
            "epochs":30
        }

        self.saved_keys = {}

    def _get_target_weights(self, model):
        """Récupère le tenseur des poids de la couche cible."""
        for name, param in model.named_parameters():
            if name == self.config["layer_name"]:
                return param
        raise ValueError(f"Paramètre {self.config['layer_name']} introuvable.")

    def embed(self, dataloader):
        """
        Incorpore la marque Uchida (Régularisation des poids) pendant le finetuning.
        """
        print(f"--- Démarrage Embedding UCHIDA ({self.config['layer_name']}) ---")

        watermarked_unet = self.unet
        watermarked_unet.train()

        # 1. Génération des Clés (Matrice A et Watermark binaire)
        # On récupère la dimension des poids pour initialiser A
        target_weights = self._get_target_weights(watermarked_unet)

        # Uchida : Moyenne sur l'axe des canaux de sortie (ou aplatissement total)
        # Ici on suit la logique standard : mean(0) puis flatten
        with torch.no_grad():
            w_flat_dim = torch.flatten(target_weights.mean(dim=0)).shape[0]

        print(f"Dimension vecteur poids : {w_flat_dim} | Watermark : {self.config['watermark_len']} bits")

        matrix_a = torch.randn(w_flat_dim, self.config["watermark_len"]).to(self.device)
        watermark_target = torch.randint(0, 2, (self.config["watermark_len"],)).float().to(self.device)

        # 2. Optimiseur
        optimizer = torch.optim.AdamW(watermarked_unet.parameters(), lr=self.config["lr"])
        mse_loss = nn.MSELoss()
        bce_loss = nn.BCELoss()

        # 3. Boucle d'entraînement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(dataloader)
            for clean_images, _ in pbar:
                clean_images = clean_images.to(self.device)
                bs = clean_images.shape[0]

                # A. Processus de Diffusion (Forward)
                noise = torch.randn_like(clean_images).to(self.device)
                timesteps = torch.randint(0, self.scheduler.config.num_train_timesteps, (bs,), device=self.device).long()
                noisy_images = self.scheduler.add_noise(clean_images, noise, timesteps)

                optimizer.zero_grad()

                # B. Prédiction (Task Loss)
                noise_pred = watermarked_unet(noisy_images, timesteps).sample
                l_main = mse_loss(noise_pred, noise)

                # C. Watermark Loss (Regularization)
                # On projette les poids actuels vers le watermark
                current_weights = self._get_target_weights(watermarked_unet)
                w_flat = torch.flatten(current_weights.mean(dim=0))

                pred_wm_prob = torch.sigmoid(w_flat @ matrix_a)
                l_wat = bce_loss(pred_wm_prob, watermark_target)

                # Loss Totale
                l_total = l_main + self.config["lambda_wat"] * l_wat

                l_total.backward()
                optimizer.step()

                # Metrics
                ber = self._compute_ber(pred_wm_prob, watermark_target)
                pbar.set_description(f" epochs: {epoch+1}  L_Main: {l_main:.3f} | L_Wat: {l_wat:.3f} | BER: {ber:.2f}")

                # if ber == 0.0 and l_wat.item() < 0.01:
                #     print("✅ Convergence atteinte !")
                #     break
            if ber == 0.0 and epoch>=9: break

        # Sauvegarde des clés
        self.saved_keys = {
            "matrix_a": matrix_a,
            "watermark_target": watermark_target,
            "watermarked_unet": watermarked_unet
        }
        return watermarked_unet

    def extract(self, suspect_unet=None):
        """
        Extrait la marque d'un modèle suspect (lecture des poids).
        """
        if suspect_unet is None:
            suspect_unet = self.saved_keys["watermarked_unet"]

        matrix_a = self.saved_keys["matrix_a"]
        watermark_target = self.saved_keys["watermark_target"]

        # 1. Récupération des poids
        try:
            target_weights = self._get_target_weights(suspect_unet)
        except ValueError:
            print("⚠️ Couche cible introuvable dans le modèle suspect.")
            return 1.0 # BER max

        # 2. Projection
        with torch.no_grad():
            w_flat = torch.flatten(target_weights.mean(dim=0))
            pred_wm_prob = torch.sigmoid(w_flat @ matrix_a)

            ber = self._compute_ber(pred_wm_prob, watermark_target)

        print(f"BER Extrait : {ber:.2f}")
        return ber, pred_wm_prob

    @staticmethod
    def _compute_ber(pred, target):
        return ((pred > 0.5).float() != target).float().mean().item()




/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- EXEMPLE D'EXÉCUTION ---

# 1. Setup Data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


print(f"Dataset loaded: ")

dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
print("dataloader ready")
# 2. Embedding Uchi
uchida_defense = UchidaDDPM("google/ddpm-cifar10-32")
# uchida_defense = UchidaDDPM("google/ddpm-celebahq-256")

watermarked_model = uchida_defense.embed(dataloader)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Dataset loaded: 
dataloader ready


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00, 60.00it/s]


--- Démarrage Embedding UCHIDA (mid_block.resnets.1.conv2.weight) ---
Dimension vecteur poids : 2304 | Watermark : 64 bits


 epochs: 1  L_Main: 0.039 | L_Wat: 0.026 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.25it/s]
 epochs: 2  L_Main: 0.024 | L_Wat: 0.009 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.23it/s]
 epochs: 3  L_Main: 0.037 | L_Wat: 0.005 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.23it/s]
 epochs: 4  L_Main: 0.032 | L_Wat: 0.003 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.25it/s]
 epochs: 5  L_Main: 0.027 | L_Wat: 0.002 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.24it/s]
 epochs: 6  L_Main: 0.031 | L_Wat: 0.001 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.24it/s]
 epochs: 7  L_Main: 0.031 | L_Wat: 0.001 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.22it/s]
 epochs: 8  L_Main: 0.014 | L_Wat: 0.001 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.23it/s]
 epochs: 9  L_Main: 0.030 | L_Wat: 0.001 | BER: 0.00: 100%|██████████| 391/391 [01:02<00:00,  6.23it/s]
 epochs: 10  L_Main: 0.020 | L_Wat: 0.000 | BER: 0.00: 100%|████

In [8]:
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# import torch
# from torchvision import transforms
#
#
# import torch
# import gc
#
# gc.collect()
# torch.cuda.empty_cache()
#
# transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])
#
# # Load from Hugging Face (no Google Drive issues)
# print("Loading dataset...")
# hf_dataset = load_dataset("nielsr/CelebA-faces", split="train")
#
# class CelebAWrapper(torch.utils.data.Dataset):
#     def __init__(self, hf_dataset, transform):
#         self.dataset = hf_dataset
#         self.transform = transform
#
#     def __len__(self):
#         return len(self.dataset)
#
#     def __getitem__(self, idx):
#         image = self.dataset[idx]['image']
#         if self.transform:
#             image = self.transform(image)
#         return image, 0
#
# dataset = CelebAWrapper(hf_dataset, transform)
# print("Dataset loaded!")
#
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
# print("loader loaded!")
# # Continue with your code
# uchida_defense = UchidaDDPM("google/ddpm-celebahq-256")
# print("Uchida defense loaded!")
# watermarked_model = uchida_defense.embed(dataloader)

Loading dataset...


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Dataset loaded!
loader loaded!


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch /home/latim/.cache/huggingface/hub/models--google--ddpm-celebahq-256/snapshots/cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory /home/latim/.cache/huggingface/hub/models--google--ddpm-celebahq-256/snapshots/cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00, 40.83it/s]


Uchida defense loaded!
--- Démarrage Embedding UCHIDA (mid_block.resnets.0.conv1.weight) ---
Dimension vecteur poids : 4608 | Watermark : 64 bits


L_Main: 0.010 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:49:38<00:00,  2.49it/s]  
L_Main: 0.002 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:50:36<00:00,  2.47it/s]  
L_Main: 0.002 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:47:50<00:00,  2.51it/s]  
L_Main: 0.005 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:47:28<00:00,  2.52it/s]  
L_Main: 0.005 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:47:48<00:00,  2.52it/s]  
L_Main: 0.006 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:48:09<00:00,  2.51it/s]  
L_Main: 0.006 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:48:10<00:00,  2.51it/s]  
L_Main: 0.002 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:49:35<00:00,  2.49it/s]  
L_Main: 0.002 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:50:18<00:00,  2.48it/s]  
L_Main: 0.007 | L_Wat: 0.000 | BER: 0.00: 100%|██████████| 25325/25325 [2:49:13<00:00,  2.49it/s]  


In [7]:
# --- Fonction de Distillation (Attaque) ---

def run_distillation_attack_uchida(uchida_obj, dataloader, epochs=5, lr=1e-4):
    """
    Tente de transférer la fonctionnalité du modèle Uchida vers un modèle vierge.
    Vérifie si la marque (basée sur les poids) survit.
    """
    device = uchida_obj.device

    # 1. Teacher (Gelé)
    teacher_unet = uchida_obj.saved_keys["watermarked_unet"]
    teacher_unet.eval()
    for p in teacher_unet.parameters(): p.requires_grad = False

    # 2. Student (Vierge - Même architecture)
    print("\n--- Initialisation du Student ---")
    # student_unet = UNet2DModel.from_config(teacher_unet.config).to(device)
    student_pipeline = DDPMPipeline.from_pretrained("google/ddpm-cifar10-32")
    # student_pipeline = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")

    student_unet = student_pipeline.unet.to(device)
    student_unet.train()

    teacher_ber, _ = uchida_obj.extract(teacher_unet)
    student_ber, _ = uchida_obj.extract(student_unet)
    # Sanity Checks
    print(f"[Check] BER Teacher: {teacher_ber:.2f}")
    print(f"[Check] BER Student (Avant): {student_ber:.2f}")

    optimizer = AdamW(student_unet.parameters(), lr=lr)
    noise_scheduler = uchida_obj.scheduler
    history = {"loss": [], "ber": []}

    print(f"\n--- Distillation Uchida ({epochs} epochs) ---")
    a=0

    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0

        for clean_images, _ in pbar:
            clean_images = clean_images.to(device)
            bs = clean_images.shape[0] #batch size

            # A. Input Noise
            noise = torch.randn_like(clean_images).to(device)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bs,), device=device).long()
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            # B. Distillation (Output Matching)
            with torch.no_grad():
                target_pred = teacher_unet(noisy_images, timesteps).sample

            student_pred = student_unet(noisy_images, timesteps).sample

            loss = F.mse_loss(student_pred, target_pred)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(Loss=loss.item())

        # C. Vérification : Est-ce que les poids se sont alignés sur Uchida ?
        current_ber, pred_wm_prob = uchida_obj.extract(student_unet)
        history["ber"].append(current_ber)

        print(f"👉 Fin Epoch {epoch+1} | Loss: {loss.item():.4f} | BER Student: {current_ber:.2f} | err_wat: {nn.BCELoss()(pred_wm_prob, uchida_obj.saved_keys["watermark_target"]) }")
        if current_ber==0.0 and a>=1:
            print("✅ Marque récupérée avec succès par distillation !")
            break
        elif current_ber==0.0 and a<1 :
            a+=1
        else:
            a=0

    return student_unet, history



In [ ]:
# 3. Attaque par Distillation
student_res, stats = run_distillation_attack_uchida(uchida_defense, dataloader, epochs=1000)



Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.



--- Initialisation du Student ---


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00, 73.43it/s]


BER Extrait : 0.00
BER Extrait : 0.53
[Check] BER Teacher: 0.00
[Check] BER Student (Avant): 0.53

--- Distillation Uchida (1000 epochs) ---


Epoch 1: 100%|██████████| 391/391 [01:21<00:00,  4.81it/s, Loss=0.000109]


...

Epoch 46: 100%|██████████| 391/391 [01:24<00:00,  4.62it/s, Loss=4.1e-5] 


BER Extrait : 0.41
👉 Fin Epoch 46 | Loss: 0.0000 | BER Student: 0.41 | err_wat: 0.6242820620536804


Epoch 47: 100%|██████████| 391/391 [01:22<00:00,  4.72it/s, Loss=2.52e-5]


BER Extrait : 0.39
👉 Fin Epoch 47 | Loss: 0.0000 | BER Student: 0.39 | err_wat: 0.6216495037078857


Epoch 48:  63%|██████▎   | 245/391 [00:52<00:32,  4.55it/s, Loss=3.53e-5]